# Técnicsa de otimização bioinspiradas
## Otimização de enxame de partículas

In [1]:
using Plots

### Função objetivo

In [2]:
function rastrigin(x, A=10)
    return A * length(x) + sum(x.^2.0 - A * cos.(2π * x))
end

rastrigin (generic function with 2 methods)

### PSO

In [17]:
function pso(cost_function, particles, domain, max_iterations, w = 0.5, c1 = 2.0, c2 = 2.0)
    dim = length(domain)
    
    # Initialization
    position = rand(particles, dim)
    for i = 1:size(position)[2]
        position[:, i] = position[:, i] * (domain[i].stop - domain[i].start) .+ domain[i].start
    end
    velocity = rand(particles, dim)
    
    best_local_fitness = 1000 * ones(particles)
    best_local = copy(position)
    
    best_global_fitness = -1000
    best_global = zeros(1, dim)
    history = zeros(particles)
    
    @gif for iter = 1:max_iterations
        # Fitness evaluation
        fitness = map(cost_function, eachrow(position))
        best_index = argmax(fitness)
        if fitness[best_index] > best_global_fitness
            best_global_fitness = fitness[best_index]
            best_global = reshape(position[best_index, :], (1, dim))
        end
        
        local_better = best_local_fitness .> fitness
        best_local[local_better, :] .= position[local_better, :]
    
        # Particle update
        velocity = w * velocity + c1 * rand() * (best_local - position) + c2 * rand() * (-position .+ best_global)
        position += velocity
        
        history[iter] = best_global_fitness
        
        x = domain[1].start:0.1:domain[1].stop
        y = domain[2].start:0.1:domain[2].stop
        contour(x, y, (x, y) -> cost_function([x, y]), c = :bluesreds)
        plot!(position[:, 1], position[:, 2], seriestype = :scatter, label = "Particles")
        plot!(best_local[:, 1], best_local[:, 2], seriestype = :scatter, label = "Best local")
        title!("Iteration #$iter")
        p1 = plot!([best_global[1]], [best_global[2]], seriestype = :scatter, label = "Best global")
        plot(1:iter, history[1:iter])
        p2 = title!("Best fitness: $best_global_fitness")
        
        plot(p1, p2, size=(1000, 400))
    end
    
    println("Best fitness: $best_global_fitness at $best_global")
end

pso (generic function with 4 methods)

In [19]:
pso(x -> -rastrigin(x), 50, [-10:10 for _ in 1:2], 50, 0.5, 2, 2)

Best fitness: -3.779945245696581e-9 at [-4.302581200759456e-6 -7.353203044856408e-7]


┌ Info: Saved animation to 
│   fn = C:\Users\vnrju\Documents\Bioinspiradas\tmp.gif
└ @ Plots C:\Users\vnrju\.julia\packages\Plots\1RWWg\src\animation.jl:114


### Resultado
<img src="tmp.gif">